In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle as pk

In [2]:
data_path_prefix = "../data/"
molecule_structures = {}
molecule_prop = {}
with open(data_path_prefix + "molecule_composition_data.pkl",'rb') as f:
    molecule_structures = pk.load(f)
with open(data_path_prefix + "molecule_prop.pkl",'rb') as f:
    molecule_prop = pk.load(f)


In [3]:

molecule_set = None
molecule_mulliken_set = None
molecule_properties_set = None

with open(data_path_prefix + "set_of_molecules.pkl",'rb') as f:
    molecule_set = pk.load(f)
with open(data_path_prefix + "molecule_prop_set.pkl",'rb') as f:
    molecule_properties_set = pk.load(f)
with open(data_path_prefix + "set_of_molecules_with_mulliken.pkl",'rb') as f:
    molecule_mulliken_set = pk.load(f)
    
training_set_molecules = molecule_set.intersection(molecule_properties_set,molecule_mulliken_set)

In [4]:
def get_input_vector(molecule):
    df = molecule_structures[molecule].drop(columns=['atom'])
    input_array = np.pad(df.values,[(0,30-df.shape[0]),(0,0)],'constant',constant_values=(0.0,0.0))
    return np.ravel(input_array)
def get_output_vector(molecule):
    df = molecule_prop[molecule]
    return np.ravel(df.drop(columns=['molecule_name','potential_energy']).values)

In [7]:
class BatchGenerator:
    def __init__(self, batch_size = 8):        
        self.molecules = np.array(list(training_set_molecules))
        self.samples = len(self.molecules)
        self.buffer = []
    def generate(self):
        for i,molecule in enumerate(self.molecules):            
            if len(self.buffer) > i:
                yield self.buffer[i]
            else:
                input_vector = get_input_vector(molecule)
                output_vector = get_output_vector(molecule)
                self.buffer.append((input_vector,output_vector))                
                yield input_vector, output_vector

In [8]:
num_epochs = 5000
batchSize = 64

batch_gen = BatchGenerator(batch_size=batchSize)

dataset = tf.data.Dataset.\
    from_generator(batch_gen.generate, (tf.float32,tf.float32),
                   output_shapes= (tf.TensorShape([240]), 
                                   tf.TensorShape([3])))                                                     
dataset = dataset.shuffle(buffer_size = batchSize*10) 
dataset = dataset.repeat(num_epochs).batch(batchSize)
dataset = dataset.prefetch(buffer_size = 2)
data_source =  dataset.make_one_shot_iterator()
X_batch, Y_batch = data_source.get_next()

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [9]:
with tf.name_scope("dnn"):
    global X_batch, loss, prediction
    hidden1 = tf.keras.layers.Dense(1024,activation='elu')(X_batch)
    hidden2 = tf.keras.layers.Dense(2048,activation='relu')(hidden1)
    dropout = tf.keras.layers.Dropout(rate=0.5)(hidden2)
    hidden3 =  tf.keras.layers.Dense(1024,activation='elu')(dropout)
    prediction = tf.keras.layers.Dense(3)(hidden2)
    
with tf.name_scope("loss"):
    loss = tf.losses.mean_squared_error(Y_batch, prediction)
init = tf.global_variables_initializer()

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


In [12]:
saver = tf.train.Saver()
with tf.Session() as sess:
    init.run()
    saver.restore(sess, "models/my_model_2080.ckpt")
    p_vector = sess.run(prediction, feed_dict={X_batch: get_input_vector("dsgdb9nsd_000001")});
    print(p_vector)

INFO:tensorflow:Restoring parameters from models/my_model_2080.ckpt


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key dnn_1/dense_1/bias not found in checkpoint
	 [[node save_2/RestoreV2 (defined at <ipython-input-12-37b18d13ea4b>:1) ]]

Caused by op 'save_2/RestoreV2', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
 